In [ ]:
!pip install pandas requests tqdm beautifulsoup4 lxml trafilatura
pip install google-cloud-bigquery-storage
pip install google-cloud-bigquery
pip install db-dtypes
pip install aiohttp aiodns beautifulsoup4
pip install langdetect
pip install transformers sentencepiece
!pip install transformers torch

SyntaxError: invalid syntax (3276572336.py, line 1)

In [9]:
!pip install --upgrade \
    pandas \
    requests \
    tqdm \
    beautifulsoup4 \
    lxml \
    trafilatura \
    aiohttp \
    aiodns \
    langdetect \
    transformers \
    sentencepiece \
    torch \
    google-cloud-bigquery \
    google-cloud-bigquery-storage \
    db-dtypes

# Corrige versões para evitar conflitos:
!pip install protobuf==6.30.0 packaging==24.2 --force-reinstall


  Using cached lxml-6.0.0-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
  Using cached protobuf-6.31.1-cp310-abi3-win_amd64.whl.metadata (593 bytes)
Using cached protobuf-6.31.1-cp310-abi3-win_amd64.whl (435 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.31.1 which is incompatible.


  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
Using cached packaging-24.2-py3-none-any.whl (65 kB)

  Attempting uninstall: protobuf

    Found existing installation: protobuf 6.31.1

    Uninstalling protobuf-6.31.1:

      Successfully uninstalled protobuf-6.31.1

   ---------------------------------------- 0/2 [protobuf]
   ---------------------------------------- 0/2 [protobuf]
   ---------------------------------------- 0/2 [protobuf]
   ---------------------------------------- 0/2 [protobuf]
   ---------------------------------------- 0/2 [protobuf]
   ---------------------------------------- 0/2 [protobuf]
  Attempting uninstall: packaging
   ---------------------------------------- 0/2 [protobuf]
    Found existing installation: packaging 24.2
   ---------------------------------------- 0/2 [protobuf]
    Uninstalling packaging-24.2:
   ---------------------------------------- 0/2 [protobuf]
      Successfully uninstalled packaging-24.2
   -------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.30.0 which is incompatible.


In [15]:
pip install googletrans==4.0.0-rc1



  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ------------------------ --------------- 0.8/1.3 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 4.5 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17459 sha256=ca79ebb2adafa7f9042f9a22464bb37c3ff72b833262ef4625a1551ed8693662
  Stored in directory: c:\users\joao altarugio\appdata\local\pip\cache\wheels\95\0f\04\b17a72024b56a60e499ce1a6313d283ed5ba332407155bee03
Successfully built googletrans

   ----------------------------------------  0/11 [rfc3986]
   --- ------------------------------------  1/11 [hyperframe]
   ------- --------------------------------  2/11 [hpack]
   ------- --------------------------------  2/11 [hpack]
  Attempting uninstall: h11
   ------- --------------------------------  2/11 [hpack]
    Found existing

  DEPRECATION: Building 'googletrans' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'googletrans'. Discussion can be found at https://github.com/pypa/pip/issues/6334
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.4.3 requires httpx>=0.25.0, but you have httpx 0.13.3 which is incompatible.
openai 1.93.0 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.30.0 which is incompatible.


In [3]:
import os
import pandas as pd
import requests
from tqdm import tqdm
from datetime import datetime, timedelta
import time
from io import StringIO
import trafilatura

In [1]:
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import os

# 🔐 CAMINHO para sua chave .json
CAMINHO_CREDENCIAL = "C:/Users/joao altarugio/Desktop/IC - Levada/gdelt-projeto-d01d5c5eb124.json"

# 💾 Pasta de destino para os arquivos CSV por ano
PASTA_SAIDA = "C:/Users/joao altarugio/Desktop/IC - Levada/DataGeral/br_news"
os.makedirs(PASTA_SAIDA, exist_ok=True)

# 🔌 Conectando ao BigQuery
credenciais = service_account.Credentials.from_service_account_file(CAMINHO_CREDENCIAL)
cliente = bigquery.Client(credentials=credenciais, project=credenciais.project_id)

# 📰 Sessão e cache para scraping otimizado
session = requests.Session()
cache = {}

def extrair_manchete(url):
    if not url or not url.startswith("http"):
        return ""
    if url in cache:
        return cache[url]
    try:
        resposta = session.get(url, timeout=3)
        soup = BeautifulSoup(resposta.text, "html.parser")
        titulo = soup.title.string if soup.title else ""
        resultado = titulo.strip()
        cache[url] = resultado
        return resultado
    except Exception:
        return ""

# 🔁 Loop por ano
for ano in range(2015, 2026):
    data_inicio = f"{ano}-01-01"
    data_fim = f"{ano+1}-01-01"

    print(f"🔄 Processando eventos de {data_inicio} até {data_fim}...")

    SQL = f"""
SELECT
  PARSE_DATE('%Y%m%d', CAST(SQLDATE AS STRING)) AS date,
  Actor1Name AS actor_1,
  Actor2Name AS actor_2,
  EventCode AS event_type,
  GoldsteinScale AS goldstein_scale,
  NumMentions AS num_articles,
  SOURCEURL AS url
FROM
  `gdelt-bq.gdeltv2.events`
WHERE
  EventCode BETWEEN '100' AND '199'
  AND (Actor1CountryCode = 'BR' OR Actor2CountryCode = 'BR' OR ActionGeo_CountryCode = 'BR')
  AND PARSE_DATE('%Y%m%d', CAST(SQLDATE AS STRING)) BETWEEN DATE('{data_inicio}') AND DATE('{data_fim}')
    """

    try:
        df = cliente.query(SQL).to_dataframe()
        df["date"] = pd.to_datetime(df["date"])

        # 🧼 Limpeza prévia das URLs
        df = df[df["url"].notnull()]
        df = df[df["url"].str.startswith("http")]
        df = df.drop_duplicates(subset="url")

        # 🔢 Top 100 eventos por dia
        top_events = (
            df.sort_values(["date", "num_articles"], ascending=[True, False])
              .groupby("date")
              .head(100)
              .reset_index(drop=True)
        )
        print(f"Total de eventos após filtro por top n/dia: {len(top_events)}")

        # ⚡ Scraping paralelo com cache
        with ThreadPoolExecutor(max_workers=20) as executor:
            top_events["headline"] = list(executor.map(extrair_manchete, top_events["url"]))

        # 🧼 Limpeza de manchetes vazias
        dados_limpos = top_events[top_events["headline"].str.strip() != ""]

        # 💾 Salvar CSV
        caminho_csv = os.path.join(PASTA_SAIDA, f"news_{ano}.csv")
        dados_limpos.to_csv(caminho_csv, index=False, encoding="utf-8")
        print(f"✅ CSV salvo: {caminho_csv}")

    except Exception as e:
        print(f"⚠️ Erro ao processar o ano {ano}: {e}")


🔄 Processando eventos de 2015-01-01 até 2016-01-01...
Total de eventos após filtro por top n/dia: 31930


C:\Users\joao altarugio\AppData\Local\Temp\ipykernel_16488\4128393153.py:31: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(resposta.text, "html.parser")


✅ CSV salvo: C:/Users/joao altarugio/Desktop/IC - Levada/DataGeral/br_news\news_2015.csv
🔄 Processando eventos de 2016-01-01 até 2017-01-01...
Total de eventos após filtro por top n/dia: 36700


C:\Users\joao altarugio\AppData\Local\Temp\ipykernel_16488\4128393153.py:31: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(resposta.text, "html.parser")


✅ CSV salvo: C:/Users/joao altarugio/Desktop/IC - Levada/DataGeral/br_news\news_2016.csv
🔄 Processando eventos de 2017-01-01 até 2018-01-01...
Total de eventos após filtro por top n/dia: 36501


...

✅ CSV salvo: C:/Users/joao altarugio/Desktop/IC - Levada/DataGeral/br_news\news_2017.csv
🔄 Processando eventos de 2018-01-01 até 2019-01-01...
Total de eventos após filtro por top n/dia: 36600


C:\Users\joao altarugio\AppData\Local\Temp\ipykernel_16488\4128393153.py:31: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(resposta.text, "html.parser")


✅ CSV salvo: C:/Users/joao altarugio/Desktop/IC - Levada/DataGeral/br_news\news_2018.csv
🔄 Processando eventos de 2019-01-01 até 2020-01-01...
Total de eventos após filtro por top n/dia: 36501


C:\Users\joao altarugio\AppData\Local\Temp\ipykernel_16488\4128393153.py:31: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(resposta.text, "html.parser")


✅ CSV salvo: C:/Users/joao altarugio/Desktop/IC - Levada/DataGeral/br_news\news_2019.csv
🔄 Processando eventos de 2020-01-01 até 2021-01-01...
Total de eventos após filtro por top n/dia: 35576
✅ CSV salvo: C:/Users/joao altarugio/Desktop/IC - Levada/DataGeral/br_news\news_2020.csv
🔄 Processando eventos de 2021-01-01 até 2022-01-01...
Total de eventos após filtro por top n/dia: 36556
✅ CSV salvo: C:/Users/joao altarugio/Desktop/IC - Levada/DataGeral/br_news\news_2021.csv
🔄 Processando eventos de 2022-01-01 até 2023-01-01...
Total de eventos após filtro por top n/dia: 36451
✅ CSV salvo: C:/Users/joao altarugio/Desktop/IC - Levada/DataGeral/br_news\news_2022.csv
🔄 Processando eventos de 2023-01-01 até 2024-01-01...
Total de eventos após filtro por top n/dia: 36563
✅ CSV salvo: C:/Users/joao altarugio/Desktop/IC - Levada/DataGeral/br_news\news_2023.csv
🔄 Processando eventos de 2024-01-01 até 2025-01-01...
Total de eventos após filtro por top n/dia: 36674
✅ CSV salvo: C:/Users/joao altarugi

# Tratamento

In [2]:
import pickle
import pandas as pd
import os

# Caminho da pasta com os arquivos
pasta = "/Users/joao altarugio/Desktop/IC - Levada/DataGeral/br_news"

# Lista para armazenar todos os DataFrames
lista_dfs = []

for ano in range(2015, 2026):  # Inclui 2025
    caminho_csv = os.path.join(pasta, f"news_{ano}.csv")
    caminho_pkl = os.path.join(pasta, f"news_{ano}.pkl")

    try:
        if os.path.exists(caminho_pkl):
            with open(caminho_pkl, "rb") as f:
                df = pickle.load(f)
        elif os.path.exists(caminho_csv):
            df = pd.read_csv(caminho_csv)
        else:
            print(f"Arquivo de {ano} não encontrado.")
            continue

        # Adiciona coluna de ano, se necessário
        df["ano"] = ano

        # Corrigir colunas, se necessário (exemplo genérico)
        df.columns = [col.strip().lower() for col in df.columns]  # padroniza nomes

        lista_dfs.append(df)

    except Exception as e:
        print(f"Erro ao carregar dados de {ano}: {e}")

# Junta tudo em um único DataFrame
df_geral = pd.concat(lista_dfs, ignore_index=True)

# Exemplo: remove linhas duplicadas, se houver
df_geral = df_geral.drop_duplicates()


print("Dados combinados e salvos com sucesso!")


Dados combinados e salvos com sucesso!


In [3]:
df_geral

,date,actor_1,actor_2,event_type,goldstein_scale,num_articles,url,headline,ano
0,2015-01-02,GANGWON,NaN,180,-9.0,10,http://www.econovill.com/news/articleView.html...,"사재혁한테 폭행당한 역도 후배, ""선수 생활 계속할지 고민"" < 미술전시 < SEO...",2015
1,2015-01-03,SCHOOL,GOVERNMENT,141,-6.5,10,"http://educacao.estadao.com.br/noticias/geral,...",Grupo invade sede da Diretoria Regional de Ens...,2015
2,2015-01-04,SCHOOL,GOVERNMENT,141,-6.5,10,http://noticias.r7.com/sao-paulo/grupo-invade-...,Grupo invade sede da Diretoria Regional de Ens...,2015
3,2015-01-04,NaN,BRAZILIAN,190,-10.0,10,http://www.franceinter.fr/emission-ailleurs-le...,Les violences policières indignent les Brésili...,2015
4,2015-01-04,BRAZIL,NaN,154,-7.2,2,http://www.noticiassin.com/2016/01/nissan-se-p...,404 - Page not found - Noticias SIN,2015
...,...,...,...,...,...,...,...,...,...
317727,2025-07-25,BRAZIL,NaN,173,-5.0,4,https://www.meionews.com/noticias/internaciona...,Brasileiro Ã© preso em Bali por trÃ¡fico de co...,2025
317728,2025-07-25,GERMANY,NaN,111,-2.0,3,https://www.scmp.com/magazines/style/people/ce...,"Who is Paul Wesley’s fiancée, Natalie Kuckenbu...",2025
317729,2025-07-25,BRAZIL,NaN,110,-2.0,2,https://www.cartacapital.com.br/carta-capital/...,O dobro ou nada – CartaCapital,2025
317730,2025-07-25,NaN,BRAZIL,112,-2.0,2,https://www.straitstimes.com/world/brazil-vp-h...,Brazil VP hails 'good' and 'fruitful' conversa...,2025


In [4]:
import pandas as pd
from langdetect import detect, DetectorFactory

DetectorFactory.seed = 0  # Para resultados consistentes

def is_en_or_pt(text):
    try:
        lang = detect(text)
        return lang in ['en', 'pt']
    except:
        return False

# Exemplo de DataFrame com headlines em vários idiomas
data = {
    'headline': [
        'This is an English headline',
        'Esta é uma manchete em português',
        'Ceci est un titre en français',
        'Dies ist eine Überschrift auf Deutsch',
        'Another English headline',
        'Mais uma manchete em português'
    ]
}

df_teste = pd.DataFrame(data)

# Aplica o filtro para manter só inglês e português
df_teste = df_teste[df_teste['headline'].apply(is_en_or_pt)].reset_index(drop=True)

print(df_teste)


                           headline
0       This is an English headline
1  Esta é uma manchete em português
2          Another English headline
3    Mais uma manchete em português


In [5]:
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0  # Para resultados consistentes

def is_en_or_pt(text):
    try:
        lang = detect(text)
        return lang in ['en', 'pt']
    except:
        return False

df_geral = df_geral[df_geral['headline'].apply(is_en_or_pt)].reset_index(drop=True)
df_geral


,date,actor_1,actor_2,event_type,goldstein_scale,num_articles,url,headline,ano
0,2015-01-03,SCHOOL,GOVERNMENT,141,-6.5,10,"http://educacao.estadao.com.br/noticias/geral,...",Grupo invade sede da Diretoria Regional de Ens...,2015
1,2015-01-04,SCHOOL,GOVERNMENT,141,-6.5,10,http://noticias.r7.com/sao-paulo/grupo-invade-...,Grupo invade sede da Diretoria Regional de Ens...,2015
2,2015-01-04,BRAZIL,NaN,154,-7.2,2,http://www.noticiassin.com/2016/01/nissan-se-p...,404 - Page not found - Noticias SIN,2015
3,2015-01-05,SPOKESMAN,REGULATOR,172,-5.0,10,http://br.advfn.com/jornal/2016/01/bolsa-de-xa...,"Bolsa de Xangai fecha em baixa de 0,26% | ADVF...",2015
4,2015-01-05,SPOKESMAN,REGULATOR,172,-5.0,4,http://www.portugaldigital.com.br/economia/ver...,404 Not Found,2015
...,...,...,...,...,...,...,...,...,...
223962,2025-07-25,MEDIA,BRAZIL,172,-5.0,4,https://www.clickorlando.com/news/politics/202...,Lula says the US has ignored Brazil’s attempts...,2025
223963,2025-07-25,BRAZIL,NaN,173,-5.0,4,https://www.meionews.com/noticias/internaciona...,Brasileiro Ã© preso em Bali por trÃ¡fico de co...,2025
223964,2025-07-25,GERMANY,NaN,111,-2.0,3,https://www.scmp.com/magazines/style/people/ce...,"Who is Paul Wesley’s fiancée, Natalie Kuckenbu...",2025
223965,2025-07-25,BRAZIL,NaN,110,-2.0,2,https://www.cartacapital.com.br/carta-capital/...,O dobro ou nada – CartaCapital,2025


In [6]:
import pandas as pd
from langdetect import detect
from googletrans import Translator

def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

translator = Translator()

def translate_if_english(text, max_words=15):
    if isinstance(text, str) and len(text.split()) <= max_words and is_english(text):
        try:
            return translator.translate(text, dest='pt').text
        except:
            return text
    return text

# Exemplo de DataFrame
data = {
    'headline': [
        'This is an English headline',
        'Esta é uma manchete em português',
        'Another English example',
        'Mais uma manchete em português',
        'This headline is definitely longer than fifteen words and should not be translated because of that'
    ]
}

df_teste2 = pd.DataFrame(data)

# Aplica a tradução só para headlines em inglês e até 15 palavras
df_teste2['headline'] = df_teste2['headline'].apply(translate_if_english)

print(df_teste2)


                                            headline
0                      Esta é uma manchete em inglês
1                   Esta é uma manchete em português
2                               Outro exemplo inglês
3                     Mais uma manchete em português
4  This headline is definitely longer than fiftee...


In [7]:
from langdetect import detect
from googletrans import Translator

def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

translator = Translator()

def translate_if_english(text, max_words=15):
    if isinstance(text, str) and len(text.split()) <= max_words and is_english(text):
        try:
            return translator.translate(text, dest='pt').text
        except:
            return text  # Se der erro na tradução, mantém o original
    return text  # Se não for inglês, mantém o original

df_geral['headline'] = df_geral['headline'].apply(translate_if_english)


#Caminho onde você quer salvar — altere se quiser salvar em outro local
caminho_arquivo = "/Users/joao altarugio/Desktop/IC - Levada/DataGeral/df_geral_02.pkl"

#Salvando o dicionário em arquivo
with open(caminho_arquivo, "wb") as f:
    pickle.dump(df_geral, f)
print(f"Arquivo salvo com sucesso em: {caminho_arquivo}")

In [1]:
import pickle
caminho_arquivo = "/Users/joao altarugio/Desktop/IC - Levada/DataGeral/df_geral_02.pkl"
with open(caminho_arquivo, "rb") as f:
   df_geral = pickle.load(f)

In [2]:
import pandas as pd

# Regex para filtrar palavras indesejadas
padrao = r'\b(403|404)\b|\[UNK\]'

# Filtra as linhas que NÃO contêm os padrões indesejados
df_geral = df_geral[~df_geral['headline'].str.contains(padrao, na=False)].copy()

# Remove duplicatas (por padrão, considera todas as colunas)
df_geral = df_geral.drop_duplicates().reset_index(drop=True)
df_geral

C:\Users\joao altarugio\AppData\Local\Temp\ipykernel_14916\3020587929.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_geral = df_geral[~df_geral['headline'].str.contains(padrao, na=False)].copy()


,date,actor_1,actor_2,event_type,goldstein_scale,num_articles,url,headline,ano
0,2015-01-03,SCHOOL,GOVERNMENT,141,-6.5,10,"http://educacao.estadao.com.br/noticias/geral,...",Grupo invade sede da Diretoria Regional de Ens...,2015
1,2015-01-04,SCHOOL,GOVERNMENT,141,-6.5,10,http://noticias.r7.com/sao-paulo/grupo-invade-...,Grupo invade sede da Diretoria Regional de Ens...,2015
2,2015-01-05,SPOKESMAN,REGULATOR,172,-5.0,10,http://br.advfn.com/jornal/2016/01/bolsa-de-xa...,"Bolsa de Xangai fecha em baixa de 0,26% | ADVF...",2015
3,2015-01-06,DEPUTY,NaN,110,-2.0,20,http://www.em.com.br/app/noticia/politica/2016...,Temer diz que não vai interferir em escolha de...,2015
4,2015-01-06,COMPANIES,NaN,100,-5.0,10,http://www.dcomercio.com.br/categoria/negocios...,Atenção necessária!|Cloudflare,2015
...,...,...,...,...,...,...,...,...,...
213129,2025-07-25,MEDIA,BRAZIL,172,-5.0,4,https://www.clickorlando.com/news/politics/202...,Lula diz que os EUA ignoraram as tentativas do...,2025
213130,2025-07-25,BRAZIL,NaN,173,-5.0,4,https://www.meionews.com/noticias/internaciona...,Brasileiro Ã© preso em Bali por trÃ¡fico de co...,2025
213131,2025-07-25,GERMANY,NaN,111,-2.0,3,https://www.scmp.com/magazines/style/people/ce...,"Who is Paul Wesley’s fiancée, Natalie Kuckenbu...",2025
213132,2025-07-25,BRAZIL,NaN,110,-2.0,2,https://www.cartacapital.com.br/carta-capital/...,O dobro ou nada – CartaCapital,2025


In [ ]:
from langdetect import detect

def is_english_and_long(text, min_words=15):
    try:
        return isinstance(text, str) and len(text.split()) > min_words and detect(text) == 'en'
    except:
        return False  # Se der erro na detecção, ignora

# Coleta os índices das linhas que satisfazem as duas condições
english_indices = df_geral[df_geral['headline'].apply(is_english_and_long)].index.tolist()


In [225]:
english_texts = df_geral.loc[english_indices, 'headline'].tolist()

In [326]:
len(english_indices)

3049

In [327]:
df_geral = df_geral.drop(index=english_indices).reset_index(drop=True)
df_geral

,date,actor_1,actor_2,event_type,goldstein_scale,num_articles,url,headline,ano
0,2015-01-03,SCHOOL,GOVERNMENT,141,-6.5,10,"http://educacao.estadao.com.br/noticias/geral,grupo-invade-sede-da-diretoria-regional-de-ensino-de-santos,10000006115",Grupo invade sede da Diretoria Regional de Ensino de Santos - Estadão,2015
1,2015-01-04,SCHOOL,GOVERNMENT,141,-6.5,10,http://noticias.r7.com/sao-paulo/grupo-invade-sede-da-diretoria-regional-de-ensino-de-santos-03012016,Grupo invade sede da Diretoria Regional de Ensino de Santos – Noticias R7,2015
2,2015-01-05,SPOKESMAN,REGULATOR,172,-5.0,10,http://br.advfn.com/jornal/2016/01/bolsa-de-xangai-fecha-em-baixa-de-0-26,"Bolsa de Xangai fecha em baixa de 0,26% | ADVFN News",2015
3,2015-01-06,DEPUTY,NaN,110,-2.0,20,"http://www.em.com.br/app/noticia/politica/2016/01/06/interna_politica,722681/temer-diz-que-nao-vai-interferir-em-escolha-de-novo-lider-da-camara.shtml",Temer diz que não vai interferir em escolha de novo líder da Câmara - Politica - Estado de Minas,2015
4,2015-01-06,COMPANIES,NaN,100,-5.0,10,http://www.dcomercio.com.br/categoria/negocios/sebrae_lanca_buscador_de_licitacoes_para_micro_e_pequena_empresa,Atenção necessária!|Cloudflare,2015
...,...,...,...,...,...,...,...,...,...
210080,2025-07-25,MEDIA,BRAZIL,172,-5.0,4,https://www.clickorlando.com/news/politics/2025/07/24/lula-says-the-us-has-ignored-brazils-attempts-to-negotiate-trumps-announced-tariff/,Lula diz que os EUA ignoraram as tentativas do Brasil de negociar a tarifa anunciada por Trump,2025
210081,2025-07-25,BRAZIL,NaN,173,-5.0,4,https://www.meionews.com/noticias/internacional/brasileiro-preso-com-cocaina-em-bali-pode-pegar-pena-de-morte-537302,Brasileiro Ã© preso em Bali por trÃ¡fico de cocaÃ­na,2025
210082,2025-07-25,GERMANY,NaN,111,-2.0,3,https://www.scmp.com/magazines/style/people/celebrities/article/3319418/who-paul-wesleys-fiancee-natalie-kuckenburg-german-brazilian-model-just-got-engaged-vampire-diaries,"Who is Paul Wesley’s fiancée, Natalie Kuckenburg? The German-Brazilian model just got engaged to The Vampire Diaries actor in Italy, has her own clothing line – and is half Wesley’s age | South China Morning Post",2025
210083,2025-07-25,BRAZIL,NaN,110,-2.0,2,https://www.cartacapital.com.br/carta-capital/o-dobro-ou-nada-2/,O dobro ou nada – CartaCapital,2025


In [ ]:
#df_geral.set_index("date", inplace=True)
#df_geral

In [4]:
!pip install protobuf==3.20.3

# Lembrar de tirar frases en ingles com mais de 15 palavras

In [5]:
import torch
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F

# Carregamento do modelo FinBERT-PT-BR
finbert_model_name = "lucas-leme/FinBERT-PT-BR"
finbert_tokenizer = AutoTokenizer.from_pretrained(finbert_model_name)
finbert_model = AutoModelForSequenceClassification.from_pretrained(finbert_model_name)

# Pré-processamento simples para português
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^\w\sáéíóúãõâêîôûàç]", "", text, flags=re.UNICODE)
    return text

# Aplica o modelo de sentimento
def sentiment_score(text):
    if not isinstance(text, str) or text.strip() == "":
        return {"PNeg": None, "PNeu": None, "PPos": None, "score": None}

    preprocessed_text = preprocess_text(text)
    inputs = finbert_tokenizer(preprocessed_text, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        outputs = finbert_model(**inputs)

    logits = outputs.logits
    probs = F.softmax(logits, dim=1).squeeze()

    return {
        "PNeg": probs[0].item(),
        "PNeu": probs[1].item(),
        "PPos": probs[2].item(),
        "score": probs[2].item() - probs[0].item()  # Positivo - Negativo
    }


In [6]:
# Aplica a função à coluna 'headline'
sentiment_results = df_geral['headline'].apply(sentiment_score)

# Expande o dicionário retornado em colunas separadas
sentiment_df = pd.DataFrame(sentiment_results.tolist())

sentiment_df

,PNeg,PNeu,PPos,score
0,0.039758,0.901313,0.058929,0.019172
1,0.040490,0.901102,0.058408,0.017918
2,0.035517,0.932744,0.031739,-0.003778
3,0.050959,0.446421,0.502620,0.451661
4,0.292568,0.229750,0.477682,0.185115
...,...,...,...,...
213129,0.026554,0.930695,0.042751,0.016196
213130,0.032355,0.925146,0.042499,0.010143
213131,0.150678,0.287107,0.562215,0.411537
213132,0.131985,0.282629,0.585386,0.453401


#Caminho onde você quer salvar — altere se quiser salvar em outro local
caminho_arquivo = "/Users/joao altarugio/Desktop/IC - Levada/DataGeral/sentiment_df_02.pkl"

#Salvando o dicionário em arquivo
with open(caminho_arquivo, "wb") as f:
    pickle.dump(sentiment_df, f)
print(f"Arquivo salvo com sucesso em: {caminho_arquivo}")

In [3]:
import pickle
caminho_arquivo = "/Users/joao altarugio/Desktop/IC - Levada/DataGeral/sentiment_df_02.pkl"

with open(caminho_arquivo, "rb") as f:
   sentiment_df = pickle.load(f)

In [ ]:
sentiment_df = sentiment_df.drop(index=english_indices).reset_index(drop=True)
sentiment_df

In [5]:
import pandas as pd

# Realiza a junção lado a lado com base no índice
df_final = pd.concat([df_geral, sentiment_df], axis=1)

df_final


,date,actor_1,actor_2,event_type,goldstein_scale,num_articles,url,headline,ano,PNeg,PNeu,PPos,score
0,2015-01-03,SCHOOL,GOVERNMENT,141,-6.5,10,"http://educacao.estadao.com.br/noticias/geral,...",Grupo invade sede da Diretoria Regional de Ens...,2015,0.039758,0.901313,0.058929,0.019172
1,2015-01-04,SCHOOL,GOVERNMENT,141,-6.5,10,http://noticias.r7.com/sao-paulo/grupo-invade-...,Grupo invade sede da Diretoria Regional de Ens...,2015,0.040490,0.901102,0.058408,0.017918
2,2015-01-05,SPOKESMAN,REGULATOR,172,-5.0,10,http://br.advfn.com/jornal/2016/01/bolsa-de-xa...,"Bolsa de Xangai fecha em baixa de 0,26% | ADVF...",2015,0.035517,0.932744,0.031739,-0.003778
3,2015-01-06,DEPUTY,NaN,110,-2.0,20,http://www.em.com.br/app/noticia/politica/2016...,Temer diz que não vai interferir em escolha de...,2015,0.050959,0.446421,0.502620,0.451661
4,2015-01-06,COMPANIES,NaN,100,-5.0,10,http://www.dcomercio.com.br/categoria/negocios...,Atenção necessária!|Cloudflare,2015,0.292568,0.229750,0.477682,0.185115
...,...,...,...,...,...,...,...,...,...,...,...,...,...
213129,2025-07-25,MEDIA,BRAZIL,172,-5.0,4,https://www.clickorlando.com/news/politics/202...,Lula diz que os EUA ignoraram as tentativas do...,2025,0.026554,0.930695,0.042751,0.016196
213130,2025-07-25,BRAZIL,NaN,173,-5.0,4,https://www.meionews.com/noticias/internaciona...,Brasileiro Ã© preso em Bali por trÃ¡fico de co...,2025,0.032355,0.925146,0.042499,0.010143
213131,2025-07-25,GERMANY,NaN,111,-2.0,3,https://www.scmp.com/magazines/style/people/ce...,"Who is Paul Wesley’s fiancée, Natalie Kuckenbu...",2025,0.150678,0.287107,0.562215,0.411537
213132,2025-07-25,BRAZIL,NaN,110,-2.0,2,https://www.cartacapital.com.br/carta-capital/...,O dobro ou nada – CartaCapital,2025,0.131985,0.282629,0.585386,0.453401


In [6]:
df_final = df_final.set_index('date')
df_final

,actor_1,actor_2,event_type,goldstein_scale,num_articles,url,headline,ano,PNeg,PNeu,PPos,score
date,,,,,,,,,,,,
2015-01-03,SCHOOL,GOVERNMENT,141,-6.5,10,"http://educacao.estadao.com.br/noticias/geral,...",Grupo invade sede da Diretoria Regional de Ens...,2015,0.039758,0.901313,0.058929,0.019172
2015-01-04,SCHOOL,GOVERNMENT,141,-6.5,10,http://noticias.r7.com/sao-paulo/grupo-invade-...,Grupo invade sede da Diretoria Regional de Ens...,2015,0.040490,0.901102,0.058408,0.017918
2015-01-05,SPOKESMAN,REGULATOR,172,-5.0,10,http://br.advfn.com/jornal/2016/01/bolsa-de-xa...,"Bolsa de Xangai fecha em baixa de 0,26% | ADVF...",2015,0.035517,0.932744,0.031739,-0.003778
2015-01-06,DEPUTY,NaN,110,-2.0,20,http://www.em.com.br/app/noticia/politica/2016...,Temer diz que não vai interferir em escolha de...,2015,0.050959,0.446421,0.502620,0.451661
2015-01-06,COMPANIES,NaN,100,-5.0,10,http://www.dcomercio.com.br/categoria/negocios...,Atenção necessária!|Cloudflare,2015,0.292568,0.229750,0.477682,0.185115
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-25,MEDIA,BRAZIL,172,-5.0,4,https://www.clickorlando.com/news/politics/202...,Lula diz que os EUA ignoraram as tentativas do...,2025,0.026554,0.930695,0.042751,0.016196
2025-07-25,BRAZIL,NaN,173,-5.0,4,https://www.meionews.com/noticias/internaciona...,Brasileiro Ã© preso em Bali por trÃ¡fico de co...,2025,0.032355,0.925146,0.042499,0.010143
2025-07-25,GERMANY,NaN,111,-2.0,3,https://www.scmp.com/magazines/style/people/ce...,"Who is Paul Wesley’s fiancée, Natalie Kuckenbu...",2025,0.150678,0.287107,0.562215,0.411537


In [7]:
import re

# Lista ampla de palavras-chave relacionadas ao Ibovespa e seu contexto
palavras_chave_ampla = [
    # Índice e ETF
    r"\bibovespa\b", r"\bibov\b", r"\bbova11\b", r"\betf\b", r"\bibvx\b",

    # Empresas
    r"\bpetrobras\b", r"\bvale\b", r"\bitau\b", r"\bita[úu]sa\b", r"\bbradesco\b", r"\bsantander\b",
    r"\bambev\b", r"\bweg\b", r"\bb3\b", r"\bbanco do brasil\b", r"\bbtg\b", r"\bmagalu\b",
    r"\blojas americanas\b", r"\bnatura\b", r"\blocaliza\b", r"\bazul\b", r"\bgol\b", r"\bcemig\b",
    r"\bsabesp\b", r"\bcpfl\b", r"\beletrobras\b", r"\bequatorial\b", r"\btelef[ôo]nica\b", r"\btim\b",

    # Macroeconomia
    r"\bselic\b", r"\bcopom\b", r"\bipca\b", r"\binflaç[aã]o\b", r"\bdeflaç[aã]o\b", r"\bpib\b",
    r"\bjuros?\b", r"\btaxa de juros?\b", r"\bdesemprego\b", r"\bfgts\b", r"\barrecadaç[aã]o\b",
    r"\bd[íi]vida p[úu]blica\b", r"\bteto de gastos\b",

    # Câmbio e moeda
    r"\bd[óo]lar(es)?\b", r"\bc[âa]mbio\b", r"\breal\b", r"\bmoedas?\b", r"\bdesvalorizaç[aã]o\b", r"\bvalorizaç[aã]o\b",

    # Mercado e movimentos
    r"\bbolsa\b", r"\bmercado\b", r"\baç[ãa]o(es)?\b", r"\balta\b", r"\bqueda\b", r"\bderretimento\b",
    r"\brecuperaç[aã]o\b", r"\brally\b", r"\bvolatilidade\b", r"\bcircuit breaker\b",
    r"\bbull market\b", r"\bbear market\b",

    # Internacional
    r"\bfed\b", r"\bfomc\b", r"\btreasury\b", r"\bwall street\b", r"\bnasdaq\b", r"\bs&p\b", r"\bdow jones\b",
    r"\brecess[ãa]o\b", r"\bcrise\b", r"\bguerra\b", r"\bconflito\b", r"\bcommodities\b",
    r"\bpetr[óo]leo\b", r"\bmin[ée]rio\b", r"\bexportaç[aã]o\b", r"\bimportaç[aã]o\b",
    r"\bchina\b", r"\beua\b", r"\bestados unidos\b", r"\beuropa\b", r"\binflaç[aã]o global\b"
]

palavras_chave_macrobr = [
    # Política monetária e fiscal
    r"\bselic\b", r"\bcopom\b", r"\bbanco central\b", r"\bmeta(s)? de inflaç[aã]o\b",
    r"\btaxa(s)? de juros?\b", r"\bjuros?\b", r"\bpol[íi]tica monet[áa]ria\b", r"\bpol[íi]tica fiscal\b",
    r"\bmeta(s)? fiscais\b", r"\bajuste fiscal\b", r"\bd[ée]ficit p[úu]blico\b", r"\bsuper[áa]vit\b",
    
    # Indicadores econômicos
    r"\bpib\b", r"\binflaç[aã]o\b", r"\bipca\b", r"\binpc\b", r"\bigp-m\b", r"\bigp-di\b", r"\bdeflaç[aã]o\b",
    r"\bpreços?\b", r"\bcusto de vida\b", r"\bdesemprego\b", r"\btaxa de ocupaç[aã]o\b",
    r"\batividade econ[oô]mica\b", r"\bprodutividade\b", r"\bn[íi]vel de confian[çc]a\b",
    
    # Orçamento, dívida e impostos
    r"\bor[çc]amento\b", r"\bLOA\b", r"\bLDO\b", r"\bplano plurianual\b",
    r"\bteto de gastos\b", r"\bd[íi]vida p[úu]blica\b", r"\bprec[áá]t[oó]rios?\b",
    r"\bcarga tribut[áa]ria\b", r"\bimpostos?\b", r"\breforma tribut[áa]ria\b",
    
    # Governo e agências
    r"\bgoverno federal\b", r"\bminist[ée]rio da economia\b", r"\btesouro nacional\b",
    r"\breceita federal\b", r"\bbndes\b", r"\bcvm\b", r"\bancine\b", r"\banatel\b", r"\banac\b",
    r"\bcongresso\b", r"\bc[âa]mara dos deputados\b", r"\bsenado\b", r"\bpl\b", r"\bpec\b",

    # Políticas públicas com impacto econômico
    r"\bbolsa fam[íi]lia\b", r"\baux[íi]lio emergencial\b", r"\bprograma de est[íi]mulo\b",
    r"\breformas?\b", r"\breforma administrativa\b", r"\breforma trabalhista\b",
    r"\bprivatizaç[aã]o\b", r"\bconcess[ãa]o\b", r"\bparceria p[úu]blico-privada\b", r"\bPPP\b",

    # Setores econômicos
    r"\bagro\b", r"\bagricultura\b", r"\bpecu[áa]ria\b", r"\bind[úu]stria\b", r"\bcom[éê]rcio\b",
    r"\bconstru[çc][ãa]o\b", r"\binfraestrutura\b", r"\benergia\b", r"\bmineraç[aã]o\b",
    r"\blog[íi]stica\b", r"\bexportaç[aã]o\b", r"\bimportaç[aã]o\b",

    # Moeda e câmbio (mantidos)
    r"\bd[óo]lar(es)?\b", r"\bc[âa]mbio\b", r"\breal\b", r"\bmoedas?\b", 
    r"\bdesvalorizaç[aã]o\b", r"\bvalorizaç[aã]o\b",

    # Outros eventos macroeconômicos
    r"\bcrise econ[oô]mica\b", r"\brecess[ãa]o\b", r"\bajuste estrutural\b", 
    r"\bincerteza econ[oô]mica\b", r"\bestagflaç[aã]o\b", r"\bchoque de oferta\b"
]

palavras_chave_politica_macro = [
    # Presidentes
    r"\bdilma( rousseff)?\b", r"\bmichel temer\b", r"\bjair bolsonaro\b", r"\bluiz in[áa]cio lula( da silva)?\b",

    # Ministros da economia
    r"\bjoaquim levy\b", r"\bnelson barbosa\b", r"\bhenrique meirelles\b", r"\bpaulo guedes\b", r"\bfernando haddad\b",

    # Banco Central
    r"\balexandre tombini\b", r"\bilan goldfajn\b", r"\broberto campos neto\b",

    # STF
    r"\bjoaquim barbosa\b", r"\bcarminha l[úu]cia\b", r"\bdias toffoli\b", r"\blewandowski\b", 
    r"\bluiz fux\b", r"\broberto barroso\b", r"\brose weber\b", r"\bgilmar mendes\b", r"\balexandre de moraes\b",

    # Outros políticos
    r"\barthur lira\b", r"\brodrigo maia\b", r"\bdavi alcolumbre\b", r"\brenan calheiros\b", 
    r"\bsimone tebet\b", r"\bfernando bezerra\b", r"\brandolfe rodrigues\b",

    # Institucional
    r"\breforma tribut[áa]ria\b", r"\breforma da previd[êe]ncia\b", r"\breforma administrativa\b",
    r"\bpec emergencial\b", r"\bplano real\b", r"\binterven[çc][ãa]o econ[oô]mica\b",
    r"\bgastos p[úu]blicos?\b", r"\bcongresso\b", r"\bsenado\b", r"\bc[âa]mara dos deputados\b",
    r"\bplanalto\b", r"\bpal[áa]cio do planalto\b"
]


padrao_amplo = re.compile("|".join(
    palavras_chave_ampla + palavras_chave_macrobr + palavras_chave_politica_macro
), flags=re.IGNORECASE)

df_final = df_final[df_final['headline'].apply(lambda x: bool(padrao_amplo.search(x)))]
df_final


,actor_1,actor_2,event_type,goldstein_scale,num_articles,url,headline,ano,PNeg,PNeu,PPos,score
date,,,,,,,,,,,,
2015-01-05,SPOKESMAN,REGULATOR,172,-5.0,10,http://br.advfn.com/jornal/2016/01/bolsa-de-xa...,"Bolsa de Xangai fecha em baixa de 0,26% | ADVF...",2015,0.035517,0.932744,0.031739,-0.003778
2015-01-13,BRAZIL,NaN,192,-9.5,4,http://www.insurancejournal.com/news/internati...,As garantias do CNP da França em negociações p...,2015,0.196207,0.109203,0.694590,0.498383
2015-01-14,CHINA,BRAZIL,130,-4.4,1,http://www.dawn.com/news/1232783,Perspectivas das economias emergentes mais pre...,2015,0.035424,0.921453,0.043123,0.007699
2015-01-19,COMPANY,NaN,141,-6.5,10,http://www.viomundo.com.br/politica/o-psdb-que...,"O PSDB, quem diria, agora ""denuncia"" desmanche...",2015,0.031467,0.926213,0.042320,0.010853
2015-01-22,BRAZILIAN,INDONESIAN,120,-4.0,2,http://www.theborneopost.com/2015/02/22/brazil...,Dilma Rousseff do Brasil recusa credenciais de...,2015,0.029844,0.912932,0.057224,0.027379
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-25,BRAZIL,AFRICA,112,-2.0,10,https://www.correiobraziliense.com.br/mundo/20...,Cresce pressÃ£o sobre Israel por crise humanit...,2025,0.033945,0.923835,0.042219,0.008274
2025-07-25,NaN,BRAZIL,120,-4.0,10,https://www.estadao.com.br/economia/aqui-ningu...,"‘Aqui ninguém põe a mão’, diz Lula, ao citar p...",2025,0.055495,0.567745,0.376760,0.321265
2025-07-25,PRESIDENT,NaN,125,-5.0,10,https://www.brasildefato.com.br/2025/07/24/nao...,"‘Não criamos esse problema, mas queremos resol...",2025,0.067383,0.362100,0.570517,0.503134


In [8]:
import numpy as np
import pandas as pd

# Garante que o índice é datetime e nomeia ele
df_final.index = pd.to_datetime(df_final.index)
df_final.index.name = "date"

# Agrupamento diário com score
daily_aggregated = df_final.groupby(df_final.index).agg(
    St=('score', 'mean'),
    sigma_t=('score', 'std')
)

# Preenche NaNs na dispersão (caso só tenha uma notícia no dia)
daily_aggregated['sigma_t'] = daily_aggregated['sigma_t'].fillna(0)

# Defasagens (lags) de 1, 2 e 3 dias para St e sigma_t
for col in ['St', 'sigma_t']:
    for lag in range(1, 4):
        daily_aggregated[f'{col}_lag{lag}'] = daily_aggregated[col].shift(lag)

# Médias móveis de 5 e 20 dias de St
daily_aggregated['MA5_St'] = daily_aggregated['St'].rolling(window=5).mean()
daily_aggregated['MA20_St'] = daily_aggregated['St'].rolling(window=20).mean()

# Aceleração do sentimento
daily_aggregated['delta_St'] = daily_aggregated['MA5_St'] - daily_aggregated['MA20_St']

# Desvios padrão rolantes de 5 e 10 dias de St
daily_aggregated['rolling_std_5_St'] = daily_aggregated['St'].rolling(window=5).std()
daily_aggregated['rolling_std_10_St'] = daily_aggregated['St'].rolling(window=10).std()

# Remove linhas com NaNs causadas pelas janelas rolantes e defasagens
daily_aggregated = daily_aggregated.dropna()

# Visualiza o resultado
print(daily_aggregated.head())


                  St   sigma_t   St_lag1   St_lag2   St_lag3  sigma_t_lag1  \
date                                                                         
2015-02-26  0.419044  0.252409  0.327052  0.120754  0.191374      0.379630   
2015-02-27  0.349777  0.313511  0.419044  0.327052  0.120754      0.252409   
2015-02-28 -0.008226  0.443796  0.349777  0.419044  0.327052      0.313511   
2015-03-01  0.026312  0.016318 -0.008226  0.349777  0.419044      0.443796   
2015-03-02  0.183904  0.260667  0.026312 -0.008226  0.349777      0.016318   

            sigma_t_lag2  sigma_t_lag3    MA5_St   MA20_St  delta_St  \
date                                                                   
2015-02-26      0.237198      0.239390  0.217509  0.210188  0.007321   
2015-02-27      0.379630      0.237198  0.281600  0.227866  0.053734   
2015-02-28      0.252409      0.379630  0.241680  0.202535  0.039145   
2015-03-01      0.313511      0.252409  0.222792  0.203466  0.019326   
2015-03-02      0.443

In [10]:
#Caminho onde você quer salvar — altere se quiser salvar em outro local
caminho_arquivo = "/Users/joao altarugio/Desktop/IC - Levada/DataGeral/daily_aggregated_02.pkl"

#Salvando o dicionário em arquivo
with open(caminho_arquivo, "wb") as f:
    pickle.dump(daily_aggregated, f)
print(f"Arquivo salvo com sucesso em: {caminho_arquivo}")

Arquivo salvo com sucesso em: /Users/joao altarugio/Desktop/IC - Levada/DataGeral/daily_aggregated_02.pkl


In [11]:
import pickle
caminho_arquivo = "/Users/joao altarugio/Desktop/IC - Levada/DataGeral/daily_aggregated_02.pkl"

with open(caminho_arquivo, "rb") as f:
   daily_aggregated = pickle.load(f)

In [13]:
daily_aggregated = daily_aggregated.dropna()
daily_aggregated

,St,sigma_t,St_lag1,St_lag2,St_lag3,sigma_t_lag1,sigma_t_lag2,sigma_t_lag3,MA5_St,MA20_St,delta_St,rolling_std_5_St,rolling_std_10_St
date,,,,,,,,,,,,,
2015-02-26,0.419044,0.252409,0.327052,0.120754,0.191374,0.379630,0.237198,0.239390,0.217509,0.210188,0.007321,0.156585,0.213704
2015-02-27,0.349777,0.313511,0.419044,0.327052,0.120754,0.252409,0.379630,0.237198,0.281600,0.227866,0.053734,0.122083,0.190130
2015-02-28,-0.008226,0.443796,0.349777,0.419044,0.327052,0.313511,0.252409,0.379630,0.241680,0.202535,0.039145,0.178541,0.197430
2015-03-01,0.026312,0.016318,-0.008226,0.349777,0.419044,0.443796,0.313511,0.252409,0.222792,0.203466,0.019326,0.198421,0.163653
2015-03-02,0.183904,0.260667,0.026312,-0.008226,0.349777,0.016318,0.443796,0.313511,0.194162,0.212118,-0.017956,0.189755,0.146440
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-21,0.138218,0.172219,0.142562,0.082541,0.207257,0.214570,0.346903,0.240308,0.160710,0.186068,-0.025357,0.059863,0.055519
2025-07-22,0.305609,0.311240,0.138218,0.142562,0.082541,0.172219,0.214570,0.346903,0.175237,0.187218,-0.011981,0.085224,0.069724
2025-07-23,0.085746,0.225727,0.305609,0.138218,0.142562,0.311240,0.172219,0.214570,0.150935,0.182146,-0.031211,0.090944,0.075588


In [14]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, GRU# captum (biblioteca de interpretação pytorch)
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, RepeatVector, TimeDistributed, Dense, Dropout
import tensorflow as tf
from sklearn.metrics import mean_squared_error
import numpy as np


# Download data for the stock (I'm using the Bovespa index as an example)
data = pd.read_csv("/Users/joao altarugio/Desktop/IC - Levada/DataGeral/Data Completo - Extended/BOVA11.csv")
data = data.drop(columns=['Adj Close', 'High', 'Low', 'Open', 'Volume'])
data

,Date,Close
0,09/04/17,45.849998
1,09/04/20,44.419998
2,09/04/22,44.990002
3,09/04/23,45.900002
4,09/04/24,46.689999
...,...,...
3948,25/04/28,131.649994
3949,25/04/29,131.860001
3950,25/04/30,131.630005
3951,25/05/02,131.850006


In [15]:
import pandas as pd

# Supondo que seu DataFrame se chama df
# 1. Converte a coluna Date para datetime
data['Date'] = pd.to_datetime(data['Date'], format="%y/%m/%d")

# 2. Filtra datas a partir de 2015-01-03
df_filtrado = data[data['Date'] >= pd.Timestamp('2015-01-03')]
df_filtrado.set_index("Date", inplace=True)
df_filtrado

,Close
Date,
2015-01-05,46.320000
2015-01-06,46.580002
2015-01-07,48.150002
2015-01-08,48.509998
2015-01-09,47.529999
...,...
2025-04-28,131.649994
2025-04-29,131.860001
2025-04-30,131.630005


#Caminho onde você quer salvar — altere se quiser salvar em outro local
caminho_arquivo = "/Users/joao altarugio/Desktop/IC - Levada/DataGeral/bova11_teste_02.pkl"

#Salvando o dicionário em arquivo
with open(caminho_arquivo, "wb") as f:
    pickle.dump(df_filtrado, f)
print(f"Arquivo salvo com sucesso em: {caminho_arquivo}")

In [16]:
import pandas as pd

# Garante que os índices estão no formato datetime
df_filtrado.index = pd.to_datetime(df_filtrado.index)
daily_aggregated.index = pd.to_datetime(daily_aggregated.index)

# Cria uma Series com os dias úteis disponíveis (fechamentos)
dias_uteis = pd.Series(df_filtrado.index, index=df_filtrado.index)

# Função para mapear cada data do daily_aggregated para o próximo dia útil
def proximo_dia_util(data):
    futuros = dias_uteis[dias_uteis >= data]
    return futuros.min() if not futuros.empty else None

# Aplica a função ao índice do daily_aggregated
daily_aggregated['mapped_date'] = daily_aggregated.index.map(proximo_dia_util)

# Remove sentimentos que não têm próximo dia útil (ex: após o último preço conhecido)
daily_aggregated = daily_aggregated.dropna(subset=['mapped_date'])

# Agrupa sentimentos que foram realocados para o mesmo dia (ex: sábado e domingo → segunda)
sentimentos_agrupados = daily_aggregated.groupby('mapped_date').mean()

# Junta os sentimentos realocados com os preços de fechamento
df_final = pd.concat([df_filtrado, sentimentos_agrupados], axis=1)

# Remove linhas com valores ausentes (dias sem sentimento ou sem fechamento)
df_final = df_final.dropna()

# Exibe o resultado
print(df_final.head())


                Close        St   sigma_t   St_lag1   St_lag2   St_lag3  \
2015-02-26  50.310001  0.419044  0.252409  0.327052  0.120754  0.191374   
2015-02-27  50.009998  0.349777  0.313511  0.419044  0.327052  0.120754   
2015-03-02  49.590000  0.067330  0.240260  0.122621  0.253532  0.365291   
2015-03-03  49.830002  0.211168  0.292292  0.183904  0.026312 -0.008226   
2015-03-04  49.070000 -0.054698  0.175204  0.211168  0.183904  0.026312   

            sigma_t_lag1  sigma_t_lag2  sigma_t_lag3    MA5_St   MA20_St  \
2015-02-26      0.379630      0.237198      0.239390  0.217509  0.210188   
2015-02-27      0.252409      0.379630      0.237198  0.281600  0.227866   
2015-03-02      0.257875      0.336572      0.315183  0.219545  0.206040   
2015-03-03      0.260667      0.016318      0.443796  0.152587  0.221308   
2015-03-04      0.292292      0.260667      0.016318  0.071692  0.198856   

            delta_St  rolling_std_5_St  rolling_std_10_St  
2015-02-26  0.007321          0.

In [322]:
#Caminho onde você quer salvar — altere se quiser salvar em outro local
caminho_arquivo = "/Users/joao altarugio/Desktop/IC - Levada/DataGeral/lagged_data_02.pkl"

#Salvando o dicionário em arquivo
with open(caminho_arquivo, "wb") as f:
    pickle.dump(df_final, f)
print(f"Arquivo salvo com sucesso em: {caminho_arquivo}")

Arquivo salvo com sucesso em: /Users/joao altarugio/Desktop/IC - Levada/DataGeral/lagged_data_02.pkl
